<h1 style="text-align: center">
<div style="color: #DD3403; font-size: 60%">Data Science DISCOVERY MicroProject</div>
<span style="">MicroProject: A DataFrame of Your Weather Forecast</span>
<div style="font-size: 60%;"><a href="https://discovery.cs.illinois.edu/microproject/dataframe-nws-weather-forecast/">https://discovery.cs.illinois.edu/microproject/dataframe-nws-weather-forecast/</a></div>
</h1>

<hr style="color: #DD3403;">

## Data API: National Weather Service (NWS) Weather Forecast

The National Weather Service allows, for free, "developers access to critical forecasts, alerts, and observations, along with other weather data."  You can find the full documentation of their API at: [https://www.weather.gov/documentation/services-web-api](https://www.weather.gov/documentation/services-web-api).

Since weather data is based on your location, you need to find the latitude and longitude of coordinate of your location!

- One quick way to do this is to pull up [Google Maps](https://maps.google.com/) and **right click** on a location.  *(Since this is NWS data, make sure your location is in the United States.)*
- The first option will be the `latitude, longitude` of the location of your click on the map.  You can click it to copy it to your clipboard!

Once you have your location, add each component to a variable below:

In [34]:
latitude = 40.11272369120086
longitude = -88.23337029665144

### Finding Your Forecast Area

The National Weather Service API provides a `/points/` endpoint that uses the `latitude,longitude` value to return a weather forecast.  The full URL for this endpoint is in the following format:

```
https://api.weather.gov/points/LATITUDE,LONGITUDE
```

This endpoint also returns a JSON that can be read using `pd.read_json`.  The `read_json` command is very similar to the `read_csv` command, except that we need to specify the "type" of JSON.

- Use `pd.read_json(...)` to use pandas's `read_json` function.
- The function requires two parameters:
    1. The URL of the JSON.  *Make sure to replace `LATITUDE` and `LONGITUDE` with your actual latitude and longitude.*
    2. For the second parameter, include the option `typ="series"`.
- For example: `pd.read_json("https://...", typ="series")`

Store the result in a variable named `pointsJSON`:

In [35]:
import pandas as pd
pointsJSON = pd.read_json("https://api.weather.gov/points/40.11272369120086,-88.23337029665144", typ="series")
pointsJSON

@context      [https://geojson.org/geojson-ld/geojson-contex...
id            https://api.weather.gov/points/40.1126999,-88....
type                                                    Feature
geometry      {'type': 'Point', 'coordinates': [-88.2334, 40...
properties    {'@id': 'https://api.weather.gov/points/40.112...
dtype: object

### Finding Your Forecast URL

The JSON you just loaded has multiple fields including two fields with data:

- `geometry`, which include the geometric area for the forecast (you supplied a single point, but weather forecasts are given for areas that can be many square miles),
- `properties`, which includes all properties about the location you requested,
- ...and a few others (`id`, `type`).

You can inspect the data further by running `df["geometry"]` or `df["properties"]`.  In the following cell, inspect the **properties** data and look for the URL that provides the **hourly forecast**.


In [36]:
pointsJSON["properties"]

{'@id': 'https://api.weather.gov/points/40.1126999,-88.2334',
 '@type': 'wx:Point',
 'cwa': 'ILX',
 'forecastOffice': 'https://api.weather.gov/offices/ILX',
 'gridId': 'ILX',
 'gridX': 96,
 'gridY': 72,
 'forecast': 'https://api.weather.gov/gridpoints/ILX/96,72/forecast',
 'forecastHourly': 'https://api.weather.gov/gridpoints/ILX/96,72/forecast/hourly',
 'forecastGridData': 'https://api.weather.gov/gridpoints/ILX/96,72',
 'observationStations': 'https://api.weather.gov/gridpoints/ILX/96,72/stations',
 'relativeLocation': {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-88.197323, 40.110126]},
  'properties': {'city': 'Urbana',
   'state': 'IL',
   'distance': {'unitCode': 'wmoUnit:m', 'value': 3081.3530245691},
   'bearing': {'unitCode': 'wmoUnit:degree_(angle)', 'value': 275}}},
 'forecastZone': 'https://api.weather.gov/zones/forecast/ILZ045',
 'county': 'https://api.weather.gov/zones/county/ILC019',
 'fireWeatherZone': 'https://api.weather.gov/zones/fire/ILZ045',


### Record Your Forecast URL

Finally, record the URL that you can use to find the hourly forecast for your location:

In [37]:
hourlyForecastURL = "https://api.weather.gov/gridpoints/ILX/96,72/forecast/hourly"
hourlyForecastURL

'https://api.weather.gov/gridpoints/ILX/96,72/forecast/hourly'

### 🔬 Microproject Checkpoint Tests 🔬

In [38]:
### TEST CASE for Data API

tada = "\N{PARTY POPPER}"
assert("/points/" in pointsJSON["id"])
assert(f"/{int(latitude)}" in pointsJSON["id"])
assert(f",{int(longitude)}" in pointsJSON["id"])
assert("/gridpoints/" in hourlyForecastURL)
assert("/forecast/hourly" in hourlyForecastURL)
print(f"{tada} Data API: All Tests Passed! {tada}")

🎉 Data API: All Tests Passed! 🎉


<hr style="color: #DD3403;">

## Puzzle 1: Loading Your Weather Forecast as a DataFrame

In the same way as you did in the beginning of this MicroProject, use `pd.read_json` to load the your weather forecast.  Store your result in the variable `forecastJSON`:

In [39]:
forecastJSON = pd.read_json("https://api.weather.gov/gridpoints/ILX/96,72/forecast/hourly",typ="series")
forecastJSON

@context      [https://geojson.org/geojson-ld/geojson-contex...
type                                                    Feature
geometry      {'type': 'Polygon', 'coordinates': [[[-88.2476...
properties    {'updated': '2023-10-06T20:45:25+00:00', 'unit...
dtype: object

### Finding the Forecast Data

Inside of the `properties` data, the `periods` data provides a **well-defined, structured** set of data that contains the forecast for every hour for your location.

Check out how it looks in the raw form:

In [40]:
# Observe the "top level" of the JSON:
forecastJSON

@context      [https://geojson.org/geojson-ld/geojson-contex...
type                                                    Feature
geometry      {'type': 'Polygon', 'coordinates': [[[-88.2476...
properties    {'updated': '2023-10-06T20:45:25+00:00', 'unit...
dtype: object

In [41]:
# Observe the "properties" data in the JSON:
forecastJSON["properties"]

{'updated': '2023-10-06T20:45:25+00:00',
 'units': 'us',
 'forecastGenerator': 'HourlyForecastGenerator',
 'generatedAt': '2023-10-06T21:26:09+00:00',
 'updateTime': '2023-10-06T20:45:25+00:00',
 'validTimes': '2023-10-06T14:00:00+00:00/P7DT11H',
 'elevation': {'unitCode': 'wmoUnit:m', 'value': 227.9904},
 'periods': [{'number': 1,
   'name': '',
   'startTime': '2023-10-06T16:00:00-05:00',
   'endTime': '2023-10-06T17:00:00-05:00',
   'isDaytime': True,
   'temperature': 54,
   'temperatureUnit': 'F',
   'temperatureTrend': None,
   'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 40},
   'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': 5},
   'relativeHumidity': {'unitCode': 'wmoUnit:percent', 'value': 61},
   'windSpeed': '20 mph',
   'windDirection': 'W',
   'icon': 'https://api.weather.gov/icons/land/day/rain_showers,40?size=small',
   'shortForecast': 'Chance Rain Showers',
   'detailedForecast': ''},
  {'number': 2,
   'name': '',
   'startTime': '2023-10-

In [42]:
# Observe the "properties"->"periods" data in the JSON:
forecastJSON["properties"]["periods"]

[{'number': 1,
  'name': '',
  'startTime': '2023-10-06T16:00:00-05:00',
  'endTime': '2023-10-06T17:00:00-05:00',
  'isDaytime': True,
  'temperature': 54,
  'temperatureUnit': 'F',
  'temperatureTrend': None,
  'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 40},
  'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': 5},
  'relativeHumidity': {'unitCode': 'wmoUnit:percent', 'value': 61},
  'windSpeed': '20 mph',
  'windDirection': 'W',
  'icon': 'https://api.weather.gov/icons/land/day/rain_showers,40?size=small',
  'shortForecast': 'Chance Rain Showers',
  'detailedForecast': ''},
 {'number': 2,
  'name': '',
  'startTime': '2023-10-06T17:00:00-05:00',
  'endTime': '2023-10-06T18:00:00-05:00',
  'isDaytime': True,
  'temperature': 53,
  'temperatureUnit': 'F',
  'temperatureTrend': None,
  'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': 17},
  'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': 4.444444444444444},
  'relativeHumidity': {'unitC

### Loading the Data at a DataFrame

Since we have **clean, structured data**, we can read it into pandas!  The code below creates a DataFrame out of the above data by using the `pd.DataFrame` command:

In [43]:
df = pd.DataFrame(forecastJSON["properties"]["periods"])
df

,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast
0,1,,2023-10-06T16:00:00-05:00,2023-10-06T17:00:00-05:00,True,54,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 40}","{'unitCode': 'wmoUnit:degC', 'value': 5}","{'unitCode': 'wmoUnit:percent', 'value': 61}",20 mph,W,https://api.weather.gov/icons/land/day/rain_sh...,Chance Rain Showers,
1,2,,2023-10-06T17:00:00-05:00,2023-10-06T18:00:00-05:00,True,53,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 17}","{'unitCode': 'wmoUnit:degC', 'value': 4.444444...","{'unitCode': 'wmoUnit:percent', 'value': 61}",20 mph,WNW,https://api.weather.gov/icons/land/day/rain_sh...,Slight Chance Rain Showers,
2,3,,2023-10-06T18:00:00-05:00,2023-10-06T19:00:00-05:00,False,51,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 8}","{'unitCode': 'wmoUnit:degC', 'value': 3.888888...","{'unitCode': 'wmoUnit:percent', 'value': 63}",16 mph,W,"https://api.weather.gov/icons/land/night/sct,8...",Partly Cloudy,
3,4,,2023-10-06T19:00:00-05:00,2023-10-06T20:00:00-05:00,False,50,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 3}","{'unitCode': 'wmoUnit:degC', 'value': 3.333333...","{'unitCode': 'wmoUnit:percent', 'value': 64}",14 mph,W,"https://api.weather.gov/icons/land/night/few,3...",Mostly Clear,
4,5,,2023-10-06T20:00:00-05:00,2023-10-06T21:00:00-05:00,False,49,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 2}","{'unitCode': 'wmoUnit:degC', 'value': 2.777777...","{'unitCode': 'wmoUnit:percent', 'value': 63}",14 mph,WNW,"https://api.weather.gov/icons/land/night/few,2...",Mostly Clear,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,152,,2023-10-12T23:00:00-05:00,2023-10-13T00:00:00-05:00,False,58,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 54}","{'unitCode': 'wmoUnit:degC', 'value': 13.33333...","{'unitCode': 'wmoUnit:percent', 'value': 93}",16 mph,E,https://api.weather.gov/icons/land/night/rain_...,Chance Rain Showers,
152,153,,2023-10-13T00:00:00-05:00,2023-10-13T01:00:00-05:00,False,57,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 54}","{'unitCode': 'wmoUnit:degC', 'value': 12.77777...","{'unitCode': 'wmoUnit:percent', 'value': 93}",16 mph,E,https://api.weather.gov/icons/land/night/rain_...,Chance Rain Showers,
153,154,,2023-10-13T01:00:00-05:00,2023-10-13T02:00:00-05:00,False,57,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 57}","{'unitCode': 'wmoUnit:degC', 'value': 12.77777...","{'unitCode': 'wmoUnit:percent', 'value': 93}",16 mph,E,https://api.weather.gov/icons/land/night/rain_...,Rain Showers Likely,
154,155,,2023-10-13T02:00:00-05:00,2023-10-13T03:00:00-05:00,False,57,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 57}","{'unitCode': 'wmoUnit:degC', 'value': 12.77777...","{'unitCode': 'wmoUnit:percent', 'value': 93}",16 mph,E,https://api.weather.gov/icons/land/night/rain_...,Rain Showers Likely,


### 🔬 Microproject Checkpoint Tests 🔬

In [44]:
### TEST CASE for Puzzle 1: Loading Your Weather Forecast as a DataFrame

tada = "\N{PARTY POPPER}"
assert("df" in vars())
assert("startTime" in df)
assert("temperature" in df)
print(f"{tada} Puzzle 1: All Tests Passed! {tada}")

🎉 Puzzle 1: All Tests Passed! 🎉


<hr style="color: #DD3403;">

## Puzzle 2: Exploring Your Weather

Now that you have the weather forecast for **your location** loaded into the DataFrame `df`, it's time to nerd out with it!

### Your Warmest Upcoming Temperature

Using your data, find the warmest temperature in the entire forecast!  Save that row as `df_warmest`:

In [45]:
df_warmest = df.nlargest(1,"temperature")
df_warmest

,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast
141,142,,2023-10-12T13:00:00-05:00,2023-10-12T14:00:00-05:00,True,68,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 53}","{'unitCode': 'wmoUnit:degC', 'value': 14.44444...","{'unitCode': 'wmoUnit:percent', 'value': 70}",17 mph,S,https://api.weather.gov/icons/land/day/rain_sh...,Chance Rain Showers,


### Your Coldest Upcoming Temperature

Using your data, find the coldest temperature in the entire forecast!  Save that row as `df_coldest`:

In [46]:
df_coldest = df.nsmallest(1,"temperature")
df_coldest

,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast
86,87,,2023-10-10T06:00:00-05:00,2023-10-10T07:00:00-05:00,True,38,F,None,"{'unitCode': 'wmoUnit:percent', 'value': 0}","{'unitCode': 'wmoUnit:degC', 'value': 1.111111...","{'unitCode': 'wmoUnit:percent', 'value': 85}",7 mph,W,"https://api.weather.gov/icons/land/day/few,0?s...",Sunny,


### Hours of Sky Conditions

The `shortForecast` column provides a brief summary of the forecasted condition of the sky for each hour.  Create a DataFrame that contains a summary of upcoming sky conditions in only two columns:

- `shortForecast`, and
- `count`, containing the number of times that `shortForecast` appears in the forecast

Save the summary in the `skySummary` variable:

In [55]:
df = df.groupby("shortForecast").agg("count").reset_index()
df["count"] = df["number"]

skySummary = pd.concat([df["shortForecast"],df["number"]])
skySummary

0           Chance Rain Showers
1                         Clear
2                  Mostly Clear
3                 Mostly Cloudy
4                  Mostly Sunny
5                 Partly Cloudy
6                  Partly Sunny
7           Rain Showers Likely
8    Slight Chance Rain Showers
9                         Sunny
0                             1
1                             1
2                             1
3                             1
4                             1
5                             1
6                             1
7                             1
8                             1
9                             1
dtype: object

### 🔬 MicroProject Checkpoint Tests 🔬

In [ ]:
### TEST CASE for Puzzle 2: Exploring Your Weather

tada = "\N{PARTY POPPER}"

assert( len(df_warmest) == 1 )
assert( df_warmest["temperature"].values[0] == max(df["temperature"]) )

assert( len(df_coldest) == 1 )
assert( df_coldest["temperature"].values[0] == min(df["temperature"]) )

assert( len(skySummary) == len(df["shortForecast"].unique()) )
pd.testing.assert_frame_equal(skySummary.sort_values(["count", "shortForecast"]).set_index("shortForecast"), pd.DataFrame(df["shortForecast"].value_counts()).reset_index().rename({"index": "shortForecast"}, axis=1).sort_values(["count", "shortForecast"]).set_index("shortForecast")), "Your skySummary is incorrect."
print(f"{tada} Puzzle 2: All Tests Passed! {tada}")


NameError: name 'skySummary' is not defined

<hr style="color: #DD3403;">

## Puzzle 3: Create a Scatter Plot

Finally, let's create a scatter plot of your forecast!

Using `df.plot.scatter`, you must specify the `x` column name and `y` column name for your scatter plot to use the command:

```
df.plot.scatter(x="x-column", y="y-column")
```

Using your weather forecast:

- Use the starting date of the forecast as your `x` value,
- Use the temperature as your `y` value,
- Create your scatter plot! :)

Here's some additional options you can add to make your chart even better:
- Add `xticks = df.startTime.values[::6]` to show only every 6th tick to get fewer x-axis labels,
- Add `rot = 90` to rotate the labels 90 degrees (vertical),
- Add `grid = True` to add gridlines,
- Add `figsize = (10, 6)` to make the figure 10" wide and 6" tall to make it larger,
- Add all of these options to get a really useful visualization! :)

In [ ]:
...

### Finally, save your graph as a variable

Just like DataFrames, graphs can be saved as a Python variable.  Return to the previous Python cell and save your graph as the variable `ax`.

Your final cell will looks something like:
```
ax = df.plot.scatter(...)
ax
```

### 🔬 MicroProject Checkpoint Tests 🔬

In [ ]:
### TEST CASE for Puzzle 3: Create a Scatter Plot

tada = "\N{PARTY POPPER}"
import matplotlib.pyplot as plt

assert( "ax" in vars() )
assert( ax.axes.get_xticklabels()[0].get_text() in df.startTime.values )
ytick = int(ax.axes.get_yticklabels()[0].get_text())
assert( ytick > -200 )
assert( ytick < 200 )

print(f"{tada} Puzzle 3: All Tests Passed! {tada}")


<hr style="color: #DD3403;">

## Submission

You're almost done!  All you need to do is to commit your lab to GitHub and run the GitHub Actions Grader:

1.  ⚠️ **Make certain to save your work.** ⚠️ To do this, go to **File => Save All**

2.  After you have saved, exit this notebook and return to https://discovery.cs.illinois.edu/microproject/dataframe-nws-weather-forecast/ and complete the section **"Commit and Grade Your Notebook"**.

3. If you see a 100% grade result on your GitHub Action, you've completed this MicroProject! 🎉